In [29]:
import pandas as pd
import numpy as np

In [30]:
df=pd.read_csv('amazon-child-review.csv')

In [31]:
df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [35]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', str(text))
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [36]:
df['review'][0]

'These flannel wipes are OK, but in my opinion not worth keeping.  I also ordered someImse Vimse Cloth Wipes-Ocean Blue-12 countwhich are larger, had a nicer, softer texture and just seemed higher quality.  I use cloth wipes for hands and faces and have been usingThirsties 6 Pack Fab Wipes, Boyfor about 8 months now and need to replace them because they are starting to get rough and have had stink issues for a while that stripping no longer handles.'

In [37]:
df['review']=df['review'].apply(preprocessor)

In [66]:
df['sentiment']=np.where(df['rating']>3 , 1,0)

In [67]:
df

,name,review,rating,sentiment
0,Planetwise Flannel Wipes,these flannel wipes are ok but in my opinion n...,3,0
1,Planetwise Wipe Pouch,it came early and was not disappointed i love ...,5,1
2,Annas Dream Full Quilt with 2 Shams,very soft and comfortable and warmer than it l...,5,1
3,Stop Pacifier Sucking without tears with Thumb...,this is a product well worth the purchase i ha...,5,1
4,Stop Pacifier Sucking without tears with Thumb...,all of my kids have cried non stop when i trie...,5,1
...,...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Sh...,such a great idea very handy to have and look ...,5,1
183527,Baby Teething Necklace for Mom Pretty Donut Sh...,this product rocks it is a great blend of func...,5,1
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),this item looks great and cool for my kids i k...,5,1
183529,"Baby Food Freezer Tray - Bacteria Resistant, B...",i am extremely happy with this product i have ...,5,1


In [68]:
import nltk
from nltk.stem.porter import PorterStemmer
porter=PorterStemmer()
def tokenizer(text):
    return[porter.stem(word) for word in text.split()]

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(strip_accents=None,
                     lowercase=False,
                     tokenizer=tokenizer,
                     norm='l2',
                     smooth_idf=True,
                     use_idf=True)
y=df.sentiment.values
x=tfidf.fit_transform(df.review)

In [71]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1,shuffle=False)

In [72]:
import pickle
from sklearn.linear_model import LogisticRegressionCV

lr=LogisticRegressionCV(cv=5,
                     scoring='accuracy',
                     random_state=1,
                     verbose=3,
                     max_iter=40).fit(x_train,y_train)
model=open('saved_model','wb')
pickle.dump(lr,model)
model.close()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/dell/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/dell/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternati

/home/dell/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/dell/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#

In [73]:
filename='saved_model'
llr=pickle.load(open(filename,'rb'))

In [74]:
llr.score(x_test,y_test)

0.8919007273816983